<a href="https://colab.research.google.com/github/Pihhot/SpeechType_Advanced/blob/main/speech_type.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>